# `vector` Size vs. Capacity #

In [1]:
#include <vector>
#include <iostream>

**NOTE, We check the data address in order to tell if new memory is allocated or not**

In [2]:
// helper function to check size and capa
template<typename T>
void query_sizes(const char *tag, const std::vector<T> &v) {
    std::cout << "vector " << tag << " has size "
        << v.size() << " and capa " << v.capacity();
}

In [3]:
// helper function to check the database addr
template<typename T>
void query_data_addr(const char *tag, const std::vector<T> &v) {
    std::cout << "vector " << tag << " currently locates at "
        << v.data();
}

In [4]:
std::vector<int> v1;
query_sizes("v1", v1);

vector v1 has size 0 and capa 0

In [5]:
// resize
v1.resize(10);

In [6]:
query_sizes("v1", v1);

vector v1 has size 10 and capa 10

In [7]:
query_data_addr("v1", v1);

vector v1 currently locates at 0x563e904752b0

In [8]:
// now push back a vector
v1.push_back(1);

In [9]:
query_sizes("v1", v1);

vector v1 has size 11 and capa 20

In [10]:
// as you can see, STL turns to be smart
// to allocate more
// as we expected, the data address has changed, thsu
// "delete" and "new" are used.
query_data_addr("v1", v1);

vector v1 currently locates at 0x563e908e8c80

In [11]:
// now with the capacity
v1.push_back(1);

In [12]:
query_sizes("v1", v1);

vector v1 has size 12 and capa 20

In [13]:
// as you can see, the capacity doen't change
// thus the memory should maintain the same
query_data_addr("v1", v1);

vector v1 currently locates at 0x563e908e8c80

In [14]:
// if we resize no larger than the capacity, no memory
// will be allocated
v1.resize(20);

In [15]:
query_data_addr("v1", v1);

vector v1 currently locates at 0x563e908e8c80

## Understand How STL Handle Capacity ##
The following part demos how STL handles capacity in vector, we compute the amortized cost.
For each data size change from `n1` to `n2` (`n2` > `n1`), we will count a `n2` operations.
This is dominated factor.

In [16]:
std::vector<int> v2;

In [17]:
int counter = 0;
std::vector<int>::size_type cur_capa = 0ul;

In [18]:
std::vector<int>::size_type additional_ops = 0ul;

In [19]:
for (int i = 0; i < 1000000; ++i) {
    v2.push_back(i);
    if (v2.capacity() > cur_capa) {
        // increment counter
        ++counter;
        cur_capa = v2.capacity();
        additional_ops += cur_capa;
    }
}

In [20]:
std::cout << "with 1000000 push backs, we ended up with "
    << counter << " memory allocation, with total operations "
    << additional_ops << ", average is "
    << additional_ops/1000000.0;

with 1000000 push backs, we ended up with 21 memory allocation, with total operations 2097151, average is 2.09715

In [21]:
// starting with a new vector
std::vector<int> v3;
counter = 0;
cur_capa = additional_ops = 0ul;

In [22]:
// 10 times larger
for (int i = 0; i < 10000000; ++i) {
    v3.push_back(i);
    if (v3.capacity() > cur_capa) {
        // increment counter
        ++counter;
        cur_capa = v3.capacity();
        additional_ops += cur_capa;
    }
}

In [23]:
std::cout << "with 10000000 push backs, we ended up with "
    << counter << " memory allocation, with total operations "
    << additional_ops << ", average is "
    << additional_ops/10000000.0;

with 10000000 push backs, we ended up with 25 memory allocation, with total operations 33554431, average is 3.35544

In [24]:
// NOTE that clear will not deallocate the vector
// we need to use shrink_to_fit after clear
v3.clear();
std::cout << "after clear, ";
query_sizes("v3", v3);
v3.shrink_to_fit();
std::cout << "\nafter shrinking, ";
query_sizes("v3", v3);

after clear, vector v3 has size 0 and capa 16777216
after shrinking, vector v3 has size 0 and capa 0

In [25]:
counter = 0;
cur_capa = additional_ops = 0ul;
// another 10 times larger
for (int i = 0; i < 100000000; ++i) {
    v3.push_back(i);
    if (v3.capacity() > cur_capa) {
        // increment counter
        ++counter;
        cur_capa = v3.capacity();
        additional_ops += cur_capa;
    }
}
std::cout << "with 100000000 push backs, we ended up with "
    << counter << " memory allocation, with total operations "
    << additional_ops << ", average is "
    << additional_ops/100000000.0;

with 100000000 push backs, we ended up with 28 memory allocation, with total operations 268435455, average is 2.68435